In [4]:
import requests 
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.keys import Keys

# 起始日和終止日

In [5]:
strDate = 20210110
endDate = 20210124

# BlockTempo 2021 (需要換年)

In [6]:
driver = webdriver.Chrome()

In [7]:
class Text2CSS(object):
    global soup_fig
    def __init__(self, Text):
        self.txt = Text
        self.block_txt = self.txt.split('<div class="content-inner ">\n')[1].split('<h3 class="p1">📍相關報導')[0]
        self.clean_txt = self.block_txt.replace('\n','').replace('\xa0','')
        self.txt_css = self.clean_txt.replace('<p class="intro_quote">','<h4 style="color: red; margin-bottom: 2.25rem">摘要： <span style="color: black">').replace('<p>','<p style="color: black; margin-bottom: 2.25rem">')
        self.intro = self.txt_css.split('</p>',1)[0]
        self.main_txt = self.txt_css.split('</p>',1)[1]
        self.css = self.css_with_fig()
    def css_with_fig(self):
        temp = self.main_txt
        for i in np.arange(len(soup_fig)):
            fig_css = '    <img src="'+re.findall(r'(data-src="?\S+)', str(soup_fig[i]))[0].strip('data-src="')+'"  style="padding-bottom: 20px;" /></p>'
            temp = temp.replace(str(soup_fig[i]), fig_css)
        return temp
    

In [8]:
class Article_CSS(Text2CSS):
    global Pic
    def ASSEMBLE(self):
        Src = pd.DataFrame({'Text' : []})
        Src = Src.append(pd.DataFrame({'Text' : [self.intro]}))
        Src = Src.append(pd.DataFrame({'Text' : ['<hr id="system-readmore" />']}))
        Src = Src.append(pd.DataFrame({'Text' : ['<p>']}))
        Src = Src.append(pd.DataFrame({'Text' : ['    &nbsp;</p>']}))
        Src = Src.append(pd.DataFrame({'Text' : ['<p>']}))
        Src = Src.append(pd.DataFrame({'Text' : ['    <img src="'+Pic+'" style="padding-bottom: 20px;" /></p>']}))
        Src = Src.append(pd.DataFrame({'Text' : [self.css]}))
        Src = Src.append(pd.DataFrame({'Text' : ['<p>']}))
        Src = Src.append(pd.DataFrame({'Text' : ['    &nbsp;</p>']}))
        Src = Src.append(pd.DataFrame({'Text' : ['<p style="font-size:20pt">']}))
        Src = Src.append(pd.DataFrame({'Text' : ['    轉貼自READ MORE ARTICLES FROM： <a href="'+url+'" style="color: navy; font-size: 20pt">'+'BlockTempo 動區'+'</a></p>']}))                                                       
        Src = Src.append(pd.DataFrame({'Text' : ['<p style="color: ; margin-bottom: 2.25rem">']})) 
        Src = Src.append(pd.DataFrame({'Text' : ['    若喜歡本文，請關注我們的臉書 Please Like our Facebook Page：<a href="http://www.facebook.com/bigdatafin/" style="color: navy;">Big Data In Finance</a></p>']}))                               
        Src = Src.append(pd.DataFrame({'Text' : ['<p style="color: ; margin-bottom: 2.25rem">']}))
        Src = Src.append(pd.DataFrame({'Text' : ['    &nbsp;</p>']}))
        
        return Src

In [9]:
i = 1 #一開始等於1
Page_count = 1

while True:
    driver.get('https://www.blocktempo.com/2021/')
    time.sleep(3)
    ix = 0
    while Page_count-ix>1 :
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.find_element_by_xpath("//a[@class='page_number'][%s]"%(Page_count-1+ix)).click() 
        ix +=1
    url = driver.find_element_by_xpath("//article[%s]//h3[@class='jeg_post_title']/a"%i).get_attribute('href')

    # Article Picture
    Pic = driver.find_elements_by_xpath('//article[%s]//img'%i)[0].get_attribute("src")

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    soup_fig = soup.find_all(['figure','img'])
    # Article Information
    Title = soup.find('title').text
    driver.find_element_by_xpath("//article[%s]//h3[@class='jeg_post_title']/a"%i).click()
    Date = driver.find_element_by_xpath("//div[@class='jeg_meta_date']/a").text
    Date = int(Date.replace('-',''))
    if Date>endDate:
            next
    if Date<strDate:
            break
    # 掀開span tag
    for match in soup.find_all('span'):
        match.unwrap()
    # 改格式
    Text = str(soup.find("div", {"class": "content-inner"}))
    CSS = Article_CSS(Text)
    
    Src = CSS.ASSEMBLE()

    
#貼文
    try:
        driver.get('https://bigdatafinance.tw/administrator')
        try:
            driver.find_element_by_id('mod-login-username').clear()
            driver.find_element_by_id('mod-login-username').send_keys('mmhft')
            driver.find_element_by_id('mod-login-password').clear()
            driver.find_element_by_id ('mod-login-password').send_keys('mmhft')
            driver.find_element_by_xpath('//button').click()
            time.sleep(3)
        except:
            time.sleep(3)    
        Error = 0
        while True:
            try:
                driver.find_element_by_xpath("//ul[@class='j-links-group nav nav-list'][1]/li[1]").click()
                break
            except:
                Error += 1
                print(str(Error)+' Times Trying to Push "新增文章"')
                time.sleep(10)
                next
        time.sleep(5)
        Error = 0
        while True:
            try:
                driver.find_element_by_xpath("//textarea[@class='cke_source cke_enable_context_menu']")
                break
            except:
                while True:
                    try:
                        driver.find_element_by_id('cke_6_label').click()
                        break
                    except:
                        Error += 1
                        try:
                            driver.find_element_by_xpath("//ul[@class='j-links-group nav nav-list'][1]/li[1]").click()
                            break
                        except:
                            Error += 1
                            print(str(Error)+' Times Trying to Push "新增文章"')
                            time.sleep(10)
                        print(str(Error)+' Times Trying to Push "原始碼"')
                        time.sleep(10)
                        next
        time.sleep(3)

        for iz in range(len(Src)):
            driver.find_element_by_xpath("//textarea[@class='cke_source cke_enable_context_menu']").send_keys(Src['Text'].iloc[iz])
            driver.find_element_by_xpath("//textarea[@class='cke_source cke_enable_context_menu']").send_keys(Keys.ENTER)
            print(str(iz+1)+'/'+str(len(Src)+1))
        time.sleep(3)

        driver.find_element_by_xpath("//input[@id='jform_title']").send_keys(Title)
        driver.find_element_by_xpath("//fieldset[@class='form-vertical']/div[@class='control-group'][2]//a").click()
        # 文章類別(務必記得更改類別，預設為Blockchain)
        driver.find_element_by_xpath("//div[@class='controls']/div[@id='jform_catid_chzn']/div[@class='chzn-drop']/div[@class='chzn-search']/input").send_keys('- Blockchain')
        driver.find_element_by_xpath("//div[@class='controls']/div[@id='jform_catid_chzn']/div[@class='chzn-drop']/div[@class='chzn-search']/input").send_keys(Keys.ENTER)                                  
        driver.find_element_by_xpath("//button[@class='btn btn-small button-apply btn-success'] ").click()
    except:
        next
    if i == 10 : 
        Page_count += 1
        i = 0                                                               
    i+=1
    print('Finished: Article'+str((Page_count-1)*10+i-1))



1/16
2/16
3/16
4/16
5/16
6/16
Finished: Article1
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
Finished: Article2
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
Finished: Article3
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
Finished: Article4
1/16
2/16
3/16
4/16
5/16
6/16
Finished: Article5
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
Finished: Article6
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
Finished: Article7
1/16
2/16
3/16
4/16
5/16
6/16
Finished: Article8
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
Finished: Article9
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
Finished: Article10
1/16
2/16
3/16
4/16
5/16
6/16
Finished: Article11
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
Finished: Article12
1

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@class='page_number'][3]"}
  (Session info: chrome=87.0.4280.141)


In [ ]:
##testing###
i = 2 #一開始等於1
Page_count = 1

driver.get('https://www.blocktempo.com/2020/')
time.sleep(3)
ix = 0

url = driver.find_element_by_xpath("//article[%s]//h3[@class='jeg_post_title']/a"%i).get_attribute('href')

    # Article Picture
Pic = driver.find_elements_by_xpath('//article[%s]//img'%i)[0].get_attribute("src")

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
soup_fig = soup.find_all(['figure','img'])
    # Article Information
Title = soup.find('title').text
driver.find_element_by_xpath("//article[%s]//h3[@class='jeg_post_title']/a"%i).click()

    # 掀開span tag
for match in soup.find_all('span'):
    match.unwrap()
    # 改格式
Text = str(soup.find("div", {"class": "content-inner"}))

soup.find("div", {"class": "content-inner"})
soup_fig = soup.find_all(['figure','img'])





'    <img src="'+re.findall(r'(data-src="?\S+)', str(soup_fig[5]))[0].strip('data-src="')+'"  style="padding-bottom: 20px;" /></p>'

Src['Text'].iloc[6].replace(str(soup_fig[5]), '    <img src="'+re.findall(r'(data-src="?\S+)', str(soup_fig[5]))[0].strip('data-src="')+'"  style="padding-bottom: 20px;" /></p>')

str(soup_fig[5])